In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
%matplotlib inline
import pandas as pd

import random

/Users/bumho/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import json
from pprint import pprint

def getGameJSON(path):
    with open(path) as data_file:    
        data = json.load(data_file)
    return data
        
data = getGameJSON('GSWvsCLE.json')


In [3]:
# read in the court png file
court = plt.imread("fullcourt.png")

headers = ["team_id", "player_id", "x_loc", "y_loc", 
           "radius"]

def playerDFtoList(frame):
    xList = frame['x_loc'].tolist()
    yList = frame['y_loc'].tolist()
    bothList = []
    for i in range(len(xList)):
        bothList.append([ xList[i], yList[i] ])
    return np.array(bothList)

In [4]:
def getMoments(data):
    player_moments = []
    for event in data['events']:
        for moment in event['moments']:
            for player in moment[5]:
                player_moments.append(player)

    df = pd.DataFrame(player_moments, columns=headers)
    return df


In [5]:
min_5 = 25*60*6*11
index = np.arange(0, min_5/11, 1)

def dist(data, ball):
    data["distanceToBall"] = np.sqrt((data["x_loc"] -ball["x_loc"])**2+ (data["y_loc"] -ball["y_loc"])**2)

def hasBall(data):
    data["hasBall"] = data["distanceToBall"].apply(lambda x: 1 if x < 2.5 else 0)
    
def transform(l, ball):
    ball.reset_index(drop=True, inplace =True)    
    result = []
    for player in l:
        player.reset_index(drop=True, inplace =True)
        dist(player, ball)
        hasBall(player)
        result.append(player[(player.hasBall == 1)][["player_id", "team_id", "x_loc", "y_loc"]])
    
    df_res = pd.concat(result)
    pos = df_res.sort_index()
    idx = np.unique(pos.index, return_index=True)[1]
    pos= pos.iloc[idx]

    pos = pos.reindex(index, fill_value=0)
    return pos

def removeRepeats(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]        

In [6]:
# Get moments from the zeroth event
player_moments = []
endsOfEvents = []
counter = -1
for event in data['events']:
    for moment in event['moments']:
        counter += 1
        for player in moment[5]:
            player_moments.append(player)
    endsOfEvents.append(counter)

df = pd.DataFrame(player_moments, columns=headers)
df = df.head(min_5)
endsOfEvents = removeRepeats(endsOfEvents)

In [7]:
# Get specific player's movements
ball = df[df.player_id==-1]

steph = df[df.player_id==201939]
klay = df[df.player_id==202691]
bogut = df[df.player_id==101106]
green = df[df.player_id==203110]
barnes = df[df.player_id==203084]

lebron = df[df.player_id==2544]
smith = df[df.player_id==2747]
love = df[df.player_id==201567]
irving = df[df.player_id==202681]
mozgov = df[df.player_id==202389]

# convert them to numpy arrays
ballA = playerDFtoList(ball)

stephA = playerDFtoList(steph)
klayA = playerDFtoList(klay)
bogutA = playerDFtoList(bogut)
greenA = playerDFtoList(green)
barnesA = playerDFtoList(barnes)

lebronA = playerDFtoList(lebron)
smithA = playerDFtoList(smith)
loveA = playerDFtoList(love)
irvingA = playerDFtoList(irving)
mozgovA = playerDFtoList(mozgov)

#Get ball radiuses
radii = ball['radius'].tolist()

In [8]:
l = [steph,klay,bogut ,green ,barnes ,lebron,smith,love,irving,mozgov]

pos = transform(l, ball)
a_pos = playerDFtoList(pos)

/Users/bumho/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bumho/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
# Get the indexes that overlap
skipRanges = []
# Only get the first 17 indices b/c we're only doing the first 5 minutes of the game
for eventEndIndex in endsOfEvents[:18]:
    lastLocation = ballA[eventEndIndex]
#     print '       Event end at '+str(eventEndIndex)+':',lastLocation
    for i in range(eventEndIndex+1,len(ballA[eventEndIndex+1:])):
        if (ballA[i] == lastLocation).all():
#             print 'matched location at '+str(i)+':',ballA[i]
            skipRanges.append([eventEndIndex, i])
            break
print 'Ranges to skip:',skipRanges

Ranges to skip: [[149, 299], [922, 1360], [1360, 1510], [1885, 2410], [2410, 2560], [2885, 3360], [3360, 3510], [3881, 4031]]


In [10]:
def distance(ball):
    basket1 = [5.32, 24.8]
    basket2 = [88.55, 24.8]
    halfcourt = 47.0
    
    ball['cavsHoop'] = np.sqrt((basket1[0] - ball['x_loc'])**2 + (basket1[1] - ball['y_loc'])**2)
    ball['warriorsHoop'] = np.sqrt((basket2[0] - ball['x_loc'])**2 + (basket2[1] - ball['y_loc'])**2)
    
distance(ball)
pos

/Users/bumho/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bumho/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,player_id,team_id,x_loc,y_loc
0,202681,1610612739,27.87412,45.48980
1,202681,1610612739,27.79347,45.43106
2,202681,1610612739,27.71793,45.36438
3,202681,1610612739,27.61261,45.33913
4,202681,1610612739,27.51496,45.30448
5,202681,1610612739,27.42254,45.26554
6,202681,1610612739,27.33418,45.22266
7,202681,1610612739,27.25215,45.17668
8,202681,1610612739,27.13954,45.15537
9,202681,1610612739,27.03458,45.13076


In [11]:
pos = pd.concat([pos, ball[["cavsHoop", "warriorsHoop"]]], axis=1)

In [12]:
dic = {201939 : "stephen curry",
202691 : "klay thompson",
101106 : "andrew bogut",
203110 : "draymond green",
203084 : "harrison barnes",
2544 : "lebron james",
2747 : "j.r. smith",
201567 : "kevin love",
202681 : "kyrie irving",
202389 : "timofey mozgov"}
team = {"1610612744":"Warriors",
"1610612739": "Cavaliers"}

In [13]:
def shot_dist(dist):
    if dist < 8:
        return "less than 8"
    elif dist < 16:
        return "8-16"
    elif dist < 24:
        return "16-24"
    else:
        return "24+"
    
def shot_decide(dist):
    if dist == 0:
        return 0
    elif dist < 8:
        if random.randint(0, 4) == 0:
            return "else"
        else:
            return "layup"
    else:
        return "jump"
    
def addParameters(data):
    data = data.copy()
    a = data["cavsHoop"][data["team_id"] == 1610612739]
    b= data["warriorsHoop"][data["team_id"] == 1610612744]
#     add distanceToBasket
    data["distanceToBasket"] = pd.concat([a,b]).reindex(index, fill_value=0)
    
    shot_dist_c = pd.get_dummies(data["distanceToBasket"].apply(shot_dist))
    
    shot_decide_c = pd.get_dummies(data["distanceToBasket"].apply(shot_decide))
    
    data["player"] = data["player_id"].apply(lambda x: 0 if x == 0 else dic[x])
    
    
    
#     return pd.concat([data, shot_dist_c,shot_decide_c], axis =1)
    return pd.concat([data,shot_decide_c], axis =1)
#     return shot_dist_c
    

In [19]:
final = addParameters(pos)
final

,player_id,team_id,x_loc,y_loc,cavsHoop,warriorsHoop,distanceToBasket,player,0,else,jump,layup
0,202681,1610612739,27.87412,45.48980,31.118304,63.353148,31.118304,kyrie irving,0,0,1,0
1,202681,1610612739,27.79347,45.43106,30.948569,63.424316,30.948569,kyrie irving,0,0,1,0
2,202681,1610612739,27.71793,45.36438,30.765752,63.520513,30.765752,kyrie irving,0,0,1,0
3,202681,1610612739,27.61261,45.33913,30.571150,63.641289,30.571150,kyrie irving,0,0,1,0
4,202681,1610612739,27.51496,45.30448,30.366090,63.786162,30.366090,kyrie irving,0,0,1,0
5,202681,1610612739,27.42254,45.26554,30.151955,63.954666,30.151955,kyrie irving,0,0,1,0
6,202681,1610612739,27.33418,45.22266,30.145333,63.955410,30.145333,kyrie irving,0,0,1,0
7,202681,1610612739,27.25215,45.17668,30.125077,63.957434,30.125077,kyrie irving,0,0,1,0
8,202681,1610612739,27.13954,45.15537,30.090595,63.960475,30.090595,kyrie irving,0,0,1,0
9,202681,1610612739,27.03458,45.13076,30.041336,63.964256,30.041336,kyrie irving,0,0,1,0


In [15]:
curry=0
thompson=0
bogut=0
green=0
barnes=0
james=0
smith=0
love=0
irving=0
timofey=0
model_dic = {201939 : curry,
202691 : thompson,
101106 : bogut,
203110 : green,
203084 : barnes,
2544 : james,
2747 : smith,
201567 : love,
202681 : irving,
202389 :timofey }

In [16]:
import shotpredictor

for player_id in dic:
    print dic[player_id]
    model_dic[player_id] = shotpredictor.predictor(dic[player_id], "2014")

lebron james
andrew bogut
stephen curry
timofey mozgov
draymond green
kyrie irving
j.r. smith
harrison barnes
klay thompson
kevin love


In [23]:
final["pct"] = 0.0
# array = final[['16-24', '24+', '8-16', 'less than 8', 'else', 'jump', 'layup']].values
array = final[['distanceToBasket', 'else', 'jump', 'layup']].values
for i in range(1000):
    if final["player_id"][i] == 0:
        final["pct"][i] = 0
    else:
        final["pct"][i] = model_dic[final["player_id"][i]].predict_proba(array[i])[0][1]
    
# model_dic[201939].predict_proba(final[['16-24', '24+', '8-16', 'less than 8', 'else', 'jump', 'layup']].values[0])

/Users/bumho/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/bumho/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bumho/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/bumho/anaconda/lib/pyt

In [24]:
final["pct"]

0       0.455772
1       0.454988
2       0.454144
3       0.453245
4       0.452299
5       0.451311
6       0.451280
7       0.451187
8       0.451028
9       0.450801
10      0.450503
11      0.450131
12      0.449684
13      0.449159
14      0.448553
15      0.447865
16      0.447092
17      0.446232
18      0.445284
19      0.444246
20      0.443117
21      0.441895
22      0.440580
23      0.439172
24      0.437671
25      0.000000
26      0.000000
27      0.000000
28      0.000000
29      0.000000
          ...   
8970    0.000000
8971    0.000000
8972    0.000000
8973    0.000000
8974    0.000000
8975    0.000000
8976    0.000000
8977    0.000000
8978    0.000000
8979    0.000000
8980    0.000000
8981    0.000000
8982    0.000000
8983    0.000000
8984    0.000000
8985    0.000000
8986    0.000000
8987    0.000000
8988    0.000000
8989    0.000000
8990    0.000000
8991    0.000000
8992    0.000000
8993    0.000000
8994    0.000000
8995    0.000000
8996    0.000000
8997    0.0000

In [31]:
l =[]
set(final["pct"].tolist())

{0.0,
 0.34009772071796596,
 0.39495735888086747,
 0.4046539379353748,
 0.41025782605383959}

In [25]:
ex = {}
ex["Ball"] = ballA.tolist()

ex["Curry"] = stephA.tolist()
ex["Thompson"] = klayA.tolist()
ex["Bogut"] = bogutA.tolist()
ex["Green"] = greenA.tolist()
ex["Barnes"] = barnesA.tolist()


ex["James"] = lebronA.tolist()
ex["Smith"] = smithA.tolist()
ex["Love"] = loveA.tolist()
ex["Irving"] = irvingA.tolist()
ex["Mozgov"] = mozgovA.tolist()

ex["radius"] = radii

ex["pos"] = a_pos.tolist()

ex["pct"] = final["pct"].tolist()

# print ex['steph'][148:152]
# print ex['steph'][299:301]

# Rebuild the dictionary with skip ranges
for player in ex:
    # Loop through the skipRanges, going in reverse so we don't mess up indices
    for thisRange in reversed(skipRanges):
        # Delete the indices to skip
        del ex[player][thisRange[0]:thisRange[1]]

In [26]:
with open("./website/public/dump.json", "w") as outfile:
    json.dump(ex, outfile)